Ravy LEON FOUN LIN - Thomas LAHAIE

# Jupyter-Notebook des figures

In [1]:
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import toml 
import numpy
import matplotlib
import plotly.express as px

Definition du path 

In [11]:
path = "../../db/metagenomics.sqlite"

## Widgets

In [3]:
import ipywidgets as widgets
from ipywidgets import Layout

district = widgets.SelectMultiple(
    options = ['porter','ekeley'],
    value = ['porter', 'ekeley'],
    description='Building',
    disabled=False,
    layout = Layout(width='50%', height='80px', display='flex')
)

In [4]:
district_treemap = widgets.SelectMultiple(
    options = ['porter','ekeley'],
    value = ['porter', 'ekeley'],
    description='Building',
    disabled=False,
    layout = Layout(width='50%', height='80px', display='flex')
)

In [5]:
district_treemap = widgets.SelectMultiple(
    options = ['porter','ekeley'],
    value = ['porter', 'ekeley'],
    description='Building',
    disabled=False,
    layout = Layout(width='50%', height='80px', display='flex')
)

In [6]:
place_treemap = widgets.Dropdown(
    options=[('door out', 'door out'), ('door in', 'door in'), ('faucet handles', 'faucet handles'),('stall out','stall out'),
            ('soap dispenser','soap dispenser'),('sink floor','sink floor')],
    value='door out',
    description='Place:',
)

In [7]:
plot_tr_sb = widgets.Dropdown(
    options=[('sunburn', 'sunburn'), ('treemap', 'treemap')],
    value='sunburn',
    description='plot:',
)

In [8]:
color_pca = widgets.Dropdown(
    options=[('gender', 'gender'), ('SAMPLE_ID', 'SAMPLE_ID'), ('building', 'buidling'),('surface','surface'),
            ('floor','floor')],
    value='gender',
    description='color_pca:',
)

### Premier barplot

In [12]:
def premier_plot(path):
         con = sqlite3.connect(path)
         cur = con.cursor()
         command = f"""
         SELECT c.abs_count, s.building, s.surface
         FROM Samples s NATURAL JOIN Counts c
         """
         res = cur.execute(command)
         result = res.fetchall()
         con.commit()
         con.close()


         dict_city = {}
         for item in result :
                  name = item[1]
                  if name not in dict_city.keys():
                           dict_city[name] = float(item[0])
                  else :
                           dict_city[name] += float(item[0])

         df = pd.DataFrame({'Proportion':list(dict_city.values()),'Building_prelevement':list(dict_city.keys())})
         
         fig = px.bar(df,x='Building_prelevement',y='Proportion',title="Proportion d'individus en general dans chaque batiment.",color='Proportion',color_continuous_scale='rdylbu')

         return fig

In [13]:
premier_plot(path)

### Second barplot

In [14]:
def update_plot(district):
    dict_fin={}
    
    
    if len(district) == 2 :
        selection_building = district

    else :
        selection_building = (district[0],district[0])
            
    con = sqlite3.connect(path)
    cur = con.cursor()
    command = f"""
        SELECT c.abs_count, s.building, s.surface
        FROM Samples s NATURAL JOIN Counts c
        WHERE s.building = '{selection_building[0]}' OR s.building = '{selection_building[1]}'
    """
    res = cur.execute(command)
    result = res.fetchall()
    con.commit()
    con.close()
    
    dict_elem = {}
    
    for item in result :
        name = item[1]+'_'+item[2]
        if name not in dict_elem.keys() :
            dict_elem[name] = float(item[0])
        else :
            dict_elem[name] += float(item[0])
    
    dict_fin = dict(sorted(dict_elem.items(), key=lambda x:x[1]))
    
    df_barplot = pd.DataFrame({'lieu':[item for item in dict_fin.keys()] ,
                          'proportion':list(dict_fin.values())})
    
    fig = px.bar(df_barplot, x='lieu', y='proportion',color='proportion', color_continuous_scale='PuRd',
             title="Proportion individus retrouver dans chaque environnement.",labels={"lieu":"Environnement de prélévement","proportion":"Proportion"})
    fig.show()

In [15]:
widgets.interactive(update_plot, district = district)

interactive(children=(SelectMultiple(description='Building', index=(0, 1), layout=Layout(display='flex', heigh…

### Treemap/Sunburst

In [18]:
def treemap_pltly(district,place,plot):    
    command1 = f"""
            SELECT t.phylum, t.class, t._order, SUM(c.abs_count) as count
            FROM Taxa t NATURAL JOIN Counts as c NATURAL JOIN Samples s
            WHERE s.building = '{district[0]}' AND s.surface = '{place}'
            group by t.phylum, t.class, t._order
        """

    requete = pd.read_sql(command1,sqlite3.connect(path))
    
    if plot == 'sunburn' :
        fig = px.sunburst(requete.fillna(''),path=['class','_order'],values='count',color='class')
    else :
        fig = px.treemap(requete.fillna(''),path=['class','_order'],values='count',color='class')

    fig.show()


In [19]:
widgets.interactive(treemap_pltly, district = district_treemap,place = place_treemap, plot = plot_tr_sb)

interactive(children=(SelectMultiple(description='Building', index=(0, 1), layout=Layout(display='flex', heigh…

### ACP

In [20]:
def acp(color):
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA

    con =  sqlite3.connect(path)

    res = pd.read_sql("""
        SELECT c.rel_count, c.OTU_ID, c.SAMPLE_ID, s.gender, s.building, s.surface, s.floor
        FROM Counts c NATURAL JOIN Samples s
    """,con)

    df = res.pivot(index=['SAMPLE_ID','gender','building','surface', "floor"],values='rel_count',columns='OTU_ID')

    SS = StandardScaler()
    SS.fit(df)
    Xnorm = SS.transform(df)

    
    pca = PCA(n_components=2)
    pca.fit(df)

    X_pca = pca.transform(df)
    
    title ="ACP sur le jeu de données avec coloration par genre."
    f=px.scatter(X_pca[:,0],X_pca[:,1],color= df.index.get_level_values(color),title=title)
    f.update_layout( title_x=0.5)

    f.show()


In [21]:
widgets.interactive(acp, color = color_pca)

interactive(children=(Dropdown(description='color_pca:', options=(('gender', 'gender'), ('SAMPLE_ID', 'SAMPLE_…

### Heatmap

In [22]:
def heatmap(path) :
    from scipy.spatial.distance import euclidean
    con =  sqlite3.connect(path)

    res3 =pd.read_sql("""
    select *
    from Samples 
    """, con)


    count = pd.read_sql("""
    select rel_count, SAMPLE_ID, OTU_ID from Counts
    """, con)
    count = count.pivot(columns="SAMPLE_ID", index="OTU_ID", values = "rel_count")

    q=res3["SAMPLE_ID"].to_frame()
    qq=q.merge(q, how='cross')
    qq["distance"] = None

    for i, v in qq.iterrows():
        s1, s2 = v["SAMPLE_ID_x"], v["SAMPLE_ID_y"]
        qq.loc[i, "distance"] = euclidean(count[s1],count[s2])


    heatmap=qq.pivot(columns=["SAMPLE_ID_y"],index="SAMPLE_ID_x",values='distance')

    return px.imshow(heatmap,color_continuous_scale='inferno',title="Distance euclidienne de l'abondance relative entre échantillons.")

In [23]:
heatmap(path)